這份 Notebook 示範 OpenAI Function Calling 基本用法


In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [ ]:
import requests
import json
from pprint import pp

In [51]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)

  if response.status_code == 200 :
    return obj["choices"][0]["message"] # 改成回傳上一層 message 物件
  else :
    return obj["error"]

## 基本的完整流程

出自官方案例 https://openai.com/blog/function-calling-and-other-api-updates

## Step 1

發出 Prompt 請求，附上你有的 function 規格

In [52]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


In [53]:
messages = [{"role": "user", "content": "今天台北市的天氣如何?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = get_completion(messages, tools=tools)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_pU7eFCAOQGtREg8ul51HqzZD',
                 'type': 'function',
                 'function': {'name': 'get_current_weather',
                              'arguments': '{"location":"Taipei","unit":"celsius"}'}}]}


## Step 2

你實際呼叫 get_current_weather 方法，帶有參數 location，得到一個結果

In [54]:
# 把 AI 的回覆加到對話歷史紀錄
messages.append(response)

# 函數名稱與函數物件的對應
available_functions = {
  "get_current_weather": get_current_weather,
}

# response 若有 function_call，表示 GPT 要我呼叫函數
if response.get("tool_calls"):
  for tool_call in response.get("tool_calls"):
    # GPT 要我執行哪一個函數
    function_name = tool_call["function"]["name"]

    # 函數物件
    fuction_to_call = available_functions[function_name]

    # 擷取出函式的參數
    function_args = json.loads(tool_call["function"]["arguments"])

    # 實際呼叫函數
    function_response = fuction_to_call(
      function_args.get("location"),
      unit=function_args.get("unit"),
    )

    # 把函數的回傳結果，塞回對話紀錄，角色是 tool
    messages.append(
        {
            "tool_call_id": tool_call["id"],
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

    pp(function_response)

('{"location": "Taipei", "temperature": "72", "unit": "celsius", "forecast": '
 '["sunny", "windy"]}')


## Step 3

 再次呼叫 OpenAI API，此時 messages 裡面有三個訊息: user, assistant 和 function 結果:

In [55]:
pp(messages)

[{'role': 'user', 'content': '今天台北市的天氣如何?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_pU7eFCAOQGtREg8ul51HqzZD',
                  'type': 'function',
                  'function': {'name': 'get_current_weather',
                               'arguments': '{"location":"Taipei","unit":"celsius"}'}}]},
 {'tool_call_id': 'call_pU7eFCAOQGtREg8ul51HqzZD',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "Taipei", "temperature": "72", "unit": "celsius", '
             '"forecast": ["sunny", "windy"]}'}]


In [56]:
response = get_completion(messages)
pp(response)

{'role': 'assistant', 'content': '今天台北市的天氣為晴天，溫度為攝氏 72 度，並有微風。祝您有個愉快的一天！'}


## 換一個問題試看看

如果問題沒有需要呼叫 function 呢?

In [57]:
messages = [{"role": "user", "content": "寫一首關於天氣的詩"}]

response = get_completion(messages, tools=tools)
pp(response)

{'role': 'assistant',
 'content': '在蔚藍的天空下\n'
            '陽光灑落在大地上\n'
            '微風輕拂臉龐\n'
            '溫暖如春的懷抱\n'
            '\n'
            '雲朵悠悠飄過\n'
            '像是天空的畫家\n'
            '描繪出美麗的風景\n'
            '讓人心情愉悅\n'
            '\n'
            '雨滴輕輕落下\n'
            '清新的氣息彌漫\n'
            '大地漸漸濕潤\n'
            '生命在這片土地上繁盛\n'
            '\n'
            '無論是晴天或雨天\n'
            '天氣總是變幻莫測\n'
            '讓我們珍惜每一刻\n'
            '感受大自然的神奇與美好'}


回傳的 response 裡面就沒有 tool_calls

## Fake function 用法: 擷取 metadata

透過一個 fake function，目的是拿結構化的 function 參數

In [59]:
messages = [
  {"role": "user", "content": "台積電 2023/5/1 的法說會資料 blah blah blah blah"}
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "information_extraction",
            "description": "Extracts the relevant information from the passage.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "報告中的公司名稱",
                    },
                    "report_date": {
                        "type": "string",
                        "description": "報告中的日期"
                    },
                }
            },
        },
    }
]

# 這個 tool_choice 參數可以要求 GPT 一定要執行某個函數，預設是 auto 讓 GPT 自行判斷
tool_choice =  {"type": "function", "function": {"name": "information_extraction"}}

response = get_completion(messages, tools=tools, tool_choice=tool_choice)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_Sbfta6nhLOWkclTWnhl7Y8zk',
                 'type': 'function',
                 'function': {'name': 'information_extraction',
                              'arguments': '{"company_name":"台積電","report_date":"2023/5/1"}'}}]}


In [60]:
metadata = json.loads(response["tool_calls"][0]["function"]["arguments"])
pp(metadata)

{'company_name': '台積電', 'report_date': '2023/5/1'}


不過在 JSON mode 這功能出了之後，就算不用 function calling 我們也可以拿到 JSON 格式

所以似乎不一定要用 function calling 這招來擷取 metadata 了，看哪一種可以更節省 tokens 數

### 使用 Google Search 工具

In [ ]:
# 這是非官方的 google 爬蟲
!pip install googlesearch-python

# 若要用官方 JSON API https://developers.google.com/custom-search/v1/overview?hl=zh-tw (有 API key 需付費但有免費額度)

  Preparing metadata (setup.py) ... done
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4209 sha256=7ade7a348764de773de9ff1b764cbc8ad42e8bf0fc634de0b8484e1916fe78ed
  Stored in directory: /root/.cache/pip/wheels/98/24/e9/6c225502948c629b01cc895f86406819281ef0da385f3eb669
Successfully built googlesearch-python


In [ ]:
from googlesearch import search

In [ ]:
def google_search(keyword):
  content = ""
  for item in search(keyword, advanced=True, num_results=5, lang='zh-tw'):
    content += f"Title: {item.title}\n Description: {item.description}\n\n"
  return content

In [61]:
messages = [{"role": "user", "content": "今天台北天氣如何?"}]

tools = [
    {
        "type": "function",
        "function": {
            "name": "google_search",
            "description": "搜尋最新的資訊",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜尋關鍵字",
                    }
                },
                "required": ["keyword"],
            },
        },
    }
]

response = get_completion(messages, tools=tools)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_6JumRz9D0VKNrQeuPZqZSqua',
                 'type': 'function',
                 'function': {'name': 'google_search',
                              'arguments': '{"keyword":"台北天氣"}'}}]}


In [62]:
metadata = json.loads(response["tool_calls"][0]["function"]["arguments"])
pp(metadata)

{'keyword': '台北天氣'}


In [63]:
messages.append(response)
available_functions = {
  "google_search": google_search,
}


if response.get("tool_calls"):
  for tool_call in response.get("tool_calls"):
    function_name = tool_call["function"]["name"]
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(tool_call["function"]["arguments"])
    function_response = fuction_to_call(**function_args)

    messages.append(
        {
            "tool_call_id": tool_call["id"],
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

    pp(function_response)

('Title: 縣市預報- 臺北市\n'
 ' Description: 今日白天 晴時多雲 25 - 3077 - 86降雨機率10%舒適; 今晚明晨 多雲時晴 19 - 2566 - '
 '77降雨機率20%稍有寒意至舒適; 明日白天 陰短暫陣雨 19 - 2266 - 72降雨機率30%稍\xa0...\n'
 '\n'
 'Title: 臺北市, 台灣- 氣象預報| 地圖\n'
 ' Description: 星期, 天氣狀況, 降雨機率, 溫度. 檢視氣象預報詳細資訊 星期三, 晴朗. 降雨機率: 0%. 華氏高溫: 83°; '
 '攝氏高溫: 29°; 華氏低溫: 67°; 攝氏低溫: 20°.\n'
 '\n'
 'Title: 臺北市, 台北市, 臺灣每小時天氣\n'
 ' Description: RealFeel Shade™80°. 風西北偏西13英里/小时. 空氣品質不健康. 最大紫外線指數2 低. '
 '陣風16英里/小时. 濕度68%. 露點69° F. 雲層2%. 能見度10英里. 雲冪30000英尺\xa0...\n'
 '\n'
 'Title: 東榮, 臺北市10 天天氣預報\n'
 ' Description: 週三21 | 白天. 28°. 10%. 東北東12 km/h. 局部多雲。 高溫28°C。 10 到15 '
 '公里/小時的東北東風。 濕度69%. 紫外線指數7 (最大值11). 日出06:24. 日落17:51\xa0...\n'
 '\n'
 'Title: 臺北市, 台北市, 臺灣三日天氣預報\n'
 ' Description: 臺北市, 台北市, 臺灣Weather Forecast, with current conditions, wind, '
 'air quality, and what to expect for the next 3 days.\n'
 '\n'
 'Title: 臺北市信義區天氣預報,臺灣七日氣象溫度,降雨機率\n'
 ' Description: 昨天（20日）為多雲到晴的天氣，臺北站白天高溫29.9度，感受上溫暖稍熱，清晨低溫21.0度，日夜溫差大。\n'
 '\n'
 'Title: 氣象儀錶板\n'
 ' Description: 各地區天氣資訊 · 南港區

In [64]:
response = get_completion(messages)
pp(response)

{'role': 'assistant',
 'content': '根據氣象預報，今天台北白天為晴時多雲，氣溫介於25至30度之間，降雨機率為10%。今晚至明晨為多雲時晴，氣溫介於19至25度之間，降雨機率為20%。明天白天則有陰短暫陣雨，氣溫介於19至22度之間，降雨機率為30%。整體來說，天氣較為舒適，但請注意降雨情況。'}
